# Future Years

PyPSA-GB can model the GB power system  by solving a network constrained Linear Optimal Power Flow (LOPF) problem. This notebook shows the example application of a FES 2022.

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)

In [2]:
import pypsa
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.style.use('ggplot')
import pandas as pd
import cartopy.crs as ccrs
import data_reader_writer

## Setting up simulation

Set the required inputs for the LOPF: the start, end and year of simulation, and the timestep.

The HVDC links used as an interconnector for import and export between GB and nearby regions such as Norway, N.Ireland, France are disregarded in this optimal economic dispatch analysis.

In [3]:
df_network=pd.read_csv('LOPF_data_heat_2022/buses.csv')
cols=df_network.name.values[29:].tolist()
cols=['Timestamp']+cols
cols

['Timestamp',
 'Beauly',
 'Peterhead',
 'Errochty',
 'Denny/Bonnybridge',
 'Neilston',
 'Strathaven',
 'Torness',
 'Eccles',
 'Harker',
 'Stella West',
 'Penwortham',
 'Deeside',
 'Daines',
 'Th. Marsh/Stocksbridge',
 'Thornton/Drax/Eggborough',
 'Keadby',
 'Ratcliffe',
 'Feckenham',
 'Walpole',
 'Bramford',
 'Pelham',
 'Sundon/East Claydon',
 'Melksham',
 'Bramley',
 'London',
 'Kemsley',
 'Sellindge',
 'Lovedean',
 'S.W.Penisula']

In [4]:
heating_EDRP=pd.read_csv('REMA/domestic_EDRP/2050/scaled with normalised profiles/hourly heat demand total_withGasboilers_MW.csv')
heating_EDRP_DHN=pd.read_csv('REMA/domestic_EDRP_DHN/2050/scaled with normalised profiles/hourly heat demand total_withDHN_MW.csv')
heating_RHPP=pd.read_csv('REMA/domestic_RHPP/2050/scaled with normalised profiles/hourly heat demand total_withHPs_MW.csv')

In [5]:
heating_EDRP.columns=cols
heating_EDRP_DHN.columns=cols
heating_RHPP.columns=cols
heating_RHPP

,Timestamp,Beauly,Peterhead,Errochty,Denny/Bonnybridge,Neilston,Strathaven,Torness,Eccles,Harker,...,Bramford,Pelham,Sundon/East Claydon,Melksham,Bramley,London,Kemsley,Sellindge,Lovedean,S.W.Penisula
0,01/01/2050 00:00,363.114531,1390.942531,511.959934,4723.969618,793.453041,8791.762574,1663.298451,531.546807,966.349815,...,2711.114527,1420.830281,4172.961722,5188.368291,3714.139290,20539.70453,5202.573462,1107.317502,5111.837340,2376.984511
1,01/01/2050 01:00,363.114531,1491.085478,511.959934,4723.969618,793.453041,8791.762574,2030.144900,648.781364,1093.582686,...,3068.068995,1420.830281,4722.387915,5871.486333,4203.155377,23244.03120,5202.573462,1253.110653,5784.879059,2547.125752
2,01/01/2050 02:00,363.114531,1664.953295,671.058534,6192.008228,793.453041,8791.762574,2351.441656,751.459428,966.349815,...,3553.630699,1420.830281,4172.961722,6800.725183,4868.359219,26922.70055,6819.344807,1451.431664,5784.879059,2547.125752
3,01/01/2050 03:00,459.302456,1863.908046,755.849386,6974.392521,1171.441267,12980.016400,2648.555444,846.409246,1426.703275,...,4002.645742,1420.830281,6160.893354,6800.725183,4203.155377,30324.48835,7680.995514,1634.825693,7547.034167,3323.015897
4,01/01/2050 04:00,545.210340,2047.349856,755.849386,6974.392521,1171.441267,12980.016400,2857.998318,913.341727,1539.524336,...,4319.167577,1420.830281,6648.085424,8265.763721,5917.120003,32722.49290,8288.394456,1634.825693,8143.839694,3585.793859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,31/12/2050 19:00,1240.171493,3718.138781,1764.413251,16280.638460,3105.323106,34408.165380,5368.358491,1715.587368,5142.595754,...,5706.552817,2609.494314,4172.961722,15042.740600,4868.359219,31703.28839,7335.305440,1323.135202,9388.391177,7612.247428
8756,31/12/2050 20:00,1240.171493,4737.042448,2244.218544,20707.909950,3086.351314,34197.950680,4907.352281,1568.261807,5352.054326,...,4002.645742,2609.494314,7731.414884,15042.740600,4203.155377,31703.28839,5202.573462,1673.559413,9388.391177,4570.640711
8757,31/12/2050 21:00,1240.171493,3938.358340,2244.218544,20707.909950,3105.323106,34408.165380,5368.358491,1715.587368,3579.445601,...,3208.889291,1420.830281,7731.414884,13728.829820,4797.123594,35438.56430,7270.608673,1684.292576,10878.978390,6116.637034
8758,31/12/2050 22:00,1240.171493,6111.556800,2244.218544,20707.909950,3041.600575,33702.095410,7739.964379,2473.490760,4727.280500,...,3208.889291,2609.494314,6395.446216,23641.812540,9156.059539,26528.75769,6719.561643,1246.913122,8143.839694,8146.387780


In [6]:
total_heating_demand=heating_EDRP_DHN+heating_RHPP+heating_EDRP

In [7]:
total_heating_demand['Timestamp']=heating_EDRP['Timestamp']           # overriding the timestamp since that altered during addition


In [8]:
total_heating_demand = total_heating_demand.reset_index(drop=True)
total_heating_demand.set_index('Timestamp')

,Beauly,Peterhead,Errochty,Denny/Bonnybridge,Neilston,Strathaven,Torness,Eccles,Harker,Stella West,...,Bramford,Pelham,Sundon/East Claydon,Melksham,Bramley,London,Kemsley,Sellindge,Lovedean,S.W.Penisula
Timestamp,,,,,,,,,,,,,,,,,,,,,
01/01/2050 00:00,1116.254341,1618.971232,1085.704450,5215.497363,2864.012213,9216.767076,2570.182367,1599.059810,1489.868557,7567.817650,...,4293.799473,2805.942173,6010.080243,8447.804219,5715.048784,27281.626761,6364.001751,1422.989810,6959.040268,3986.695624
01/01/2050 01:00,1116.254341,1690.303657,1085.704450,5215.497363,2864.012213,9216.767076,2888.451998,1659.113527,1548.566172,7814.362805,...,4443.560370,2805.942173,6319.004277,8704.220680,5942.120392,29103.350141,6364.001751,1527.457445,7390.259650,3741.278575
01/01/2050 02:00,1116.254341,1865.988981,1171.011014,6620.318231,2864.012213,9216.767076,3212.018160,1764.462959,1489.868557,9999.746123,...,4932.758941,2805.942173,6010.080243,9640.949416,6611.922143,32797.511814,7831.396231,1726.503842,7390.259650,3741.278575
01/01/2050 03:00,1248.646683,2107.040734,1353.350346,7486.272446,3327.733937,13422.618623,3677.043766,2057.065470,1971.898815,11361.390918,...,5650.863284,2805.942173,8074.079538,9640.949416,5942.120392,37345.566372,8890.513861,1963.568718,9470.722313,4753.939903
01/01/2050 04:00,1940.723093,2481.119913,1353.350346,7486.272446,3327.733937,13422.618623,4694.599776,3075.245697,2513.095842,7814.362805,...,7262.437499,2805.942173,10064.517543,14327.235242,9638.149096,45260.235784,10448.266487,1963.568718,11579.025471,6141.036405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31/12/2050 19:00,11793.144141,4566.932243,8998.412080,22478.015572,37279.964914,41422.877550,11728.690576,9202.474152,15440.935812,37078.983205,...,19188.159575,9059.098076,6010.080243,38907.540803,6611.922143,85479.915353,16214.487341,2289.297437,17989.665211,25130.257944
31/12/2050 20:00,11793.144141,6458.967915,13470.022422,30325.072141,39068.934773,41583.762286,14313.131992,12640.011974,18440.050576,48552.738235,...,5650.863284,9059.098076,17983.037925,38907.540803,5942.120392,85479.915353,6364.001751,4164.998240,17989.665211,11504.448125
31/12/2050 21:00,11793.144141,4541.278097,13470.022422,30325.072141,37279.964914,41422.877550,11728.690576,9202.474152,9617.032115,46290.281937,...,7881.707675,2805.942173,17983.037925,32276.467137,10901.413660,63991.342544,12857.787115,4290.212544,13642.176359,10451.685386


In [9]:
total_heating_demand.to_csv('LOPF_data_heat_2050/total_heating_demand.csv',index=False,header=True)

In [10]:
network = pypsa.Network()
network.import_from_csv_folder('LOPF_data_heat_2050')

INFO:pypsa.components:Applying weightings to all columns of `snapshot_weightings`
INFO:pypsa.io:Imported network LOPF_data_heat_2050 has buses, generators, lines, links, loads, storage_units


A boiler at each heat nodes is integrated and added as a generator compoenent which is used as a back up heating source to the storage and the heat pump units to meet the hour by hour heat demand for the yearly operating period. The nominal capcity of the boiler is allwed to be extendable so that the optimisation algorithm will dispatch the needed optimal back up boiler capcity.

In [11]:
for i in range(29):
    network.add(
        "Generator",
        "boiler {}".format(i+1),
       bus='Heat Bus {}'.format(i+1),
       p_nom_extendable=True,
        ramp_limit_up=1,
        ramp_limit_down=1,
        efficiency=0.80,
        marginal_cost=20.0,
        carrier="heat",
    )

Links need to be scaled up to accomadate for future generation.

In [12]:
contingency_factor = 0.7
network.lines.s_max_pu *= contingency_factor

In [13]:
network.generators

,carrier,type,p_nom,bus,marginal_cost,ramp_limit_up,ramp_limit_down,p_max_pu,control,p_nom_extendable,...,committable,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
West Burton,Coal,Conventional steam,0.000000,Keadby,NaN,1.0,1.0,1.0,PQ,False,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0
Sutton Bridge CCS Gas,CCS Gas,CCS Gas,116.671400,Walpole,100.0,1.0,1.0,1.0,PQ,False,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0
Baglan Bay CCS Gas,CCS Gas,CCS Gas,74.077080,Melksham,100.0,1.0,1.0,1.0,PQ,False,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0
Severn Power CCS Gas,CCS Gas,CCS Gas,121.087540,Melksham,100.0,1.0,1.0,1.0,PQ,False,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0
Blackburn CCS Gas,CCS Gas,CCS Gas,8.547356,Penwortham,100.0,1.0,1.0,1.0,PQ,False,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
boiler 25,heat,,0.000000,Heat Bus 25,20.0,1.0,1.0,1.0,PQ,True,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0
boiler 26,heat,,0.000000,Heat Bus 26,20.0,1.0,1.0,1.0,PQ,True,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0
boiler 27,heat,,0.000000,Heat Bus 27,20.0,1.0,1.0,1.0,PQ,True,...,False,0.0,0.0,0,0,1,0,1.0,1.0,0.0


In [14]:
network.consistency_check()

## Running the optimisation

In [15]:
#network.lopf(network.snapshots, solver_name="gurobi", pyomo=False)

In [16]:
#network.model
Linopy_model=network.optimize.create_model()

C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-fix-p-ramp_limit_up` to match existing coordinates.
  warnings.warn(
C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-fix-p-ramp_limit_down` to match existing coordinates.
  warnings.warn(
C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-ext-p-ramp_limit_up` to match existing coordinates.
  warnings.warn(
C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-ext-p-ramp_limit_down` to match existing coordinates.
  warnings.warn(
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


In [17]:

#network.model.constraints.remove("Kirchhoff-Voltage-Law")    Removing the Kirchoff's Voltage Law constraints which converts to a transport model
#network.optimize(solver_name='gurobi')

In [ ]:

def remove_kvl(network, sns):
    print("KVL removed!")
    network.model.constraints.remove("Kirchhoff-Voltage-Law")
network.optimize(solver_name='gurobi',extra_functionality=remove_kvl)


C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-fix-p-ramp_limit_up` to match existing coordinates.
  warnings.warn(
C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-fix-p-ramp_limit_down` to match existing coordinates.
  warnings.warn(
C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-ext-p-ramp_limit_up` to match existing coordinates.
  warnings.warn(
C:\Users\salene\Anaconda3\envs\PyPSA-GB\lib\site-packages\linopy\model.py:607: UserWarning: Reindexing constraint `Generator-ext-p-ramp_limit_down` to match existing coordinates.
  warnings.warn(
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
INFO:linopy.model: Solve linear problem using Gu

KVL removed!


Writing variables.: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.75s/it]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-29


Read LP format model from file C:\Users\salene\AppData\Local\Temp\linopy-problem-cio5ol1g.lp
Reading time = 68.89 seconds
obj: 28039896 rows, 9075419 columns, 49938767 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28039896 rows, 9075419 columns and 49938767 nonzeros
Model fingerprint: 0xf60e814c
Coefficient statistics:
  Matrix range     [7e-01, 3e+03]
  Objective range  [1e+00, 1e+09]
  Bounds range     [1e+07, 1e+07]
  RHS range        [1e-06, 7e+12]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 25359348 rows and 1253894 columns (presolve time = 5s) ...
Presolve removed 25359348 rows and 1253894 columns (presolve time = 10s) ...
Presolve removed 25359348 rows and 2942553 columns (presolve tim

  86   1.06129031e+11  1.04001210e+11  8.65e-07 4.14e-06  1.76e+02   857s
  87   1.06100579e+11  1.04047086e+11  8.27e-07 4.14e-06  1.70e+02   869s
  88   1.06073925e+11  1.04102319e+11  7.12e-07 4.15e-06  1.63e+02   880s
  89   1.06047660e+11  1.04121110e+11  9.54e-07 5.19e-06  1.59e+02   890s
  90   1.06016514e+11  1.04158156e+11  1.08e-06 6.24e-06  1.53e+02   899s
  91   1.05995184e+11  1.04202095e+11  1.04e-06 6.24e-06  1.48e+02   910s
  92   1.05967739e+11  1.04248479e+11  1.01e-06 6.25e-06  1.42e+02   919s
  93   1.05944154e+11  1.04278204e+11  8.20e-07 7.30e-06  1.38e+02   929s
  94   1.05919741e+11  1.04300317e+11  8.20e-07 6.26e-06  1.34e+02   941s
  95   1.05902476e+11  1.04322033e+11  7.97e-07 6.26e-06  1.30e+02   953s
  96   1.05882618e+11  1.04353693e+11  7.75e-07 5.21e-06  1.26e+02   963s
  97   1.05856205e+11  1.04379820e+11  1.48e-06 6.27e-06  1.22e+02   975s
  98   1.05839791e+11  1.04396645e+11  1.44e-06 6.27e-06  1.19e+02   987s
  99   1.05817063e+11  1.04418874e+11 

In [ ]:
network.generators.p_nom_opt.div(1e3) # GW  optimised capacity of generators in GW
network.storage_units.p_nom_opt.div(1e3)  # GW 
#My_network.links_t.efficiency 
network.storage_units

In [ ]:
network.storage_units.index[120:150]
#network.storage_units.p_nom_opt[4:33].div(1e3).values

In [ ]:
network.storage_units.index[120:150]

In [ ]:
plt.figure(figsize=(19,11))
plt.bar(network.storage_units.index[:4], network.storage_units.p_nom_opt[:4].div(1e3).values)

plt.xticks(network.storage_units.index[:4], rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title('Optimal Sizing of Pumped Hydroelectric Storage')
plt.show()

In [ ]:
plt.figure(figsize=(19,11))
plt.bar(network.storage_units.index[4:33], network.storage_units.p_nom_opt[4:33].div(1e3).values)

plt.xticks(network.storage_units.index[4:33], rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title('Battrey Energy storage')
plt.show()

In [ ]:
plt.figure(figsize=(19,11))
plt.bar(network.storage_units.index[33:91], network.storage_units.p_nom_opt[33:91].div(1e3).values)

plt.xticks(network.storage_units.index[33:91], rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title('Compressed and Liquid air storage')
plt.show()

In [ ]:
plt.figure(figsize=(19,11))
plt.bar(network.storage_units.index[91:120], network.storage_units.p_nom_opt[91:120].div(1e3).values)

plt.xticks(network.storage_units.index[91:120], rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title('P2G storage')
plt.show()

In [ ]:
plt.figure(figsize=(19,11))
plt.bar(network.storage_units.index[120:150], network.storage_units.p_nom_opt[120:150].div(1e3).values)

plt.xticks(network.storage_units.index[120:150], rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title('STES')
plt.show()

## Power output by generation type

Group the generators by the carrier, and print their summed power outputs over the simulation period.

In [ ]:
year=2035

In [ ]:
p_by_carrier = network.generators_t.p.groupby(
    network.generators.carrier, axis=1).sum()

storage_by_carrier = network.storage_units_t.p.groupby(
    network.storage_units.carrier, axis=1).sum()

# to show on graph set the negative storage values to zero
storage_by_carrier[storage_by_carrier < 0] = 0

p_by_carrier = pd.concat([p_by_carrier, storage_by_carrier], axis=1)

#imp = network.links_t.p0.copy()
#imp[imp < 0] = 0
#imp['Interconnectors Import'] = imp.sum(axis=1)
#interconnector_import = imp[['Interconnectors Import']]

#p_by_carrier = pd.concat([p_by_carrier, interconnector_import], axis=1)

#exp = network.links_t.p0.copy()
#exp[exp > 0] = 0
#exp['Interconnectors Export'] = exp.sum(axis=1)
#interconnector_export = exp[['Interconnectors Export']]

# group biomass stuff
p_by_carrier['Biomass'] = (
    p_by_carrier['Biomass (dedicated)'] + p_by_carrier['Biomass (co-firing)'])

# rename the hydro bit
p_by_carrier = p_by_carrier.rename(
    columns={'Large Hydro': 'Hydro'})
p_by_carrier = p_by_carrier.rename(
    columns={'Interconnector': 'Interconnectors Import'})

generators_p_nom = network.generators.p_nom.groupby(
    network.generators.carrier).sum().sort_values()
if year > 2020:
    generators_p_nom.drop('Unmet Load', inplace=True)
generators_p_nom.drop(generators_p_nom[generators_p_nom < 50].index, inplace=True)

plt.rcParams.update({'font.size': 12})
# bar chart
plt.figure(figsize=(10,4))
plt.bar(generators_p_nom.index, generators_p_nom.values / 1000)
plt.xticks(generators_p_nom.index, rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title('Installed capacity in year ' + str(year))
plt.show()

Graph the power output of the different generation types...

In [ ]:
#cols = ["Nuclear", 'Biomass',
        #'Waste', "Oil", "Natural Gas",
       # 'Hydrogen', 'CCS Gas', 'CCS Biomass',
        #"Pumped Storage Hydroelectric", 'Hydro',
        #'Battery', 'Compressed Air', 'Liquid Air',
        #"Wind Offshore", 'Wind Onshore', 'Solar Photovoltaics',
        #'Interconnectors Import', 'Unmet Load'
       # ]
# ignore the import/export since the the highvoltage links are disrgearded

cols = ["Nuclear", 'Biomass',
        'Waste', "Oil", "Natural Gas",
        'Hydrogen', 'CCS Gas', 'CCS Biomass',
        "Pumped Storage Hydroelectric", 'Hydro',
        'Battery', 'Compressed Air', 'Liquid Air',
        "Wind Offshore", 'Wind Onshore', 'Solar Photovoltaics',
         'Unmet Load'
        ]




p_by_carrier = p_by_carrier[cols]

p_by_carrier.drop(
    (p_by_carrier.max()[p_by_carrier.max() < 50.0]).index,
    axis=1, inplace=True)

colors = {"Coal": "grey",
          "Diesel/Gas oil": "black",
          "Diesel/gas Diesel/Gas oil": "black",
          'Oil': 'black',
          'Unmet Load': 'black',
          'Anaerobic Digestion': 'green',
          'Waste': 'chocolate',
          'Sewage Sludge Digestion': 'green',
          'Landfill Gas': 'green',
          'Biomass (dedicated)': 'green',
          'Biomass (co-firing)': 'green',
          'Biomass': 'green',
          'CCS Biomass': 'darkgreen',
          'Interconnectors Import': 'pink',
          'B6 import': 'pink',
          "Sour gas": "lightcoral",
          "Natural Gas": "lightcoral",
          'CCS Gas': "lightcoral",
          'Hydrogen': "deeppink",
          "Nuclear": "orange",
          'Shoreline Wave': 'aqua',
          'Tidal Barrage and Tidal Stream': 'aqua',
          'Hydro': "turquoise",
          "Large Hydro": "turquoise",
          "Small Hydro": "turquoise",
          "Pumped Storage Hydroelectric": "darkturquoise",
          'Battery': 'lime',
          'Compressed Air': 'greenyellow',
          'Liquid Air': 'lawngreen',
          "Wind Offshore": "lightskyblue",
          'Wind Onshore': 'deepskyblue',
          'Solar Photovoltaics': 'yellow'}

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)
(p_by_carrier / 1e3).plot(
    kind="area", ax=ax, linewidth=0,
    color=[colors[col] for col in p_by_carrier.columns])

# # stacked area plot of negative values, prepend column names with '_' such that they don't appear in the legend
# (interconnector_export / 1e3).plot.area(ax=ax, stacked=True, linewidth=0.)
# # rescale the y axis
# ax.set_ylim([(interconnector_export / 1e3).sum(axis=1).min(), (p_by_carrier / 1e3).sum(axis=1).max()])

# Shrink current axis's height by 10% on the bottom
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])

# Put a legend below current axis
ax.legend(loc='upper center', bbox_to_anchor=(0.52, -0.05),
          fancybox=True, shadow=True, ncol=5)

ax.set_ylabel("GW")

ax.set_xlabel("")

## Plotting storage

Graph the pumped hydro dispatch and state of charge...

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage = network.storage_units_t.p.sum(axis=1)
state_of_charge = network.storage_units_t.state_of_charge.sum(axis=1)
p_storage.plot(label="Pumped hydro dispatch", ax=ax, linewidth=3)
state_of_charge.plot(label="State of charge", ax=ax, linewidth=3)

ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")

Let us plot the state of charge and dispatch for each type of storages separately

In [ ]:
network.storage_units_t.state_of_charge.iloc[: , :4].sum(axis=1)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage_PumpedHydorES = network.storage_units_t.p.iloc[: , :4].sum(axis=1)

state_of_charge_PumpedHydroES = network.storage_units_t.state_of_charge.iloc[: , :4].sum(axis=1)


p_storage_PumpedHydorES.plot(label="Pumped HydroElectric Storage dispatch", ax=ax, linewidth=3)
state_of_charge_PumpedHydroES.plot(label="State of charge", ax=ax, linewidth=3)
ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")


In [ ]:
network.storage_units_t.state_of_charge.iloc[: , 4:33].sum(axis=1)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage_PumpedHydorES = network.storage_units_t.p.iloc[: , 4:33].sum(axis=1)

state_of_charge_PumpedHydroES = network.storage_units_t.state_of_charge.iloc[: , 4:33].sum(axis=1)


p_storage_PumpedHydorES.plot(label="Battery Electric Storage dispatch", ax=ax, linewidth=3)
state_of_charge_PumpedHydroES.plot(label="State of charge", ax=ax, linewidth=3)
ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")


In [ ]:
network.storage_units_t.state_of_charge.iloc[: , 33:91].sum(axis=1)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage_PumpedHydorES = network.storage_units_t.p.iloc[: , 33:91].sum(axis=1)

state_of_charge_PumpedHydroES = network.storage_units_t.state_of_charge.iloc[: , 33:91].sum(axis=1)


p_storage_PumpedHydorES.plot(label="Compressed Air and Liquid Storage dispatch", ax=ax, linewidth=3)
state_of_charge_PumpedHydroES.plot(label="State of charge", ax=ax, linewidth=3)
ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")


In [ ]:
network.storage_units_t.state_of_charge.iloc[: , 91:120].sum(axis=1)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage_PumpedHydorES = network.storage_units_t.p.iloc[: , 91:120].sum(axis=1)

state_of_charge_PumpedHydroES = network.storage_units_t.state_of_charge.iloc[: , 91:120].sum(axis=1)


p_storage_PumpedHydorES.plot(label="P2G Storage dispatch", ax=ax, linewidth=3)
state_of_charge_PumpedHydroES.plot(label="State of charge", ax=ax, linewidth=3)
ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")


In [ ]:
network.storage_units_t.state_of_charge.iloc[: , 120:150].sum(axis=1)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

p_storage_PumpedHydorES = network.storage_units_t.p.iloc[: , 120:150].sum(axis=1)

state_of_charge_PumpedHydroES = network.storage_units_t.state_of_charge.iloc[: , 120:150].sum(axis=1)


p_storage_PumpedHydorES.plot(label="STES dispatch", ax=ax, linewidth=3)
state_of_charge_PumpedHydroES.plot(label="State of charge", ax=ax, linewidth=3)
ax.legend()
ax.grid()
ax.set_ylabel("MWh")
ax.set_xlabel("")

Let us add dditional constraint to the seasonal thermal energy storage and solve our model again with the conservative constraints included on Storages

## Plotting line loading

Look at the line loading stats and graph...

In [ ]:
now = network.snapshots[60]

print("With the linear load flow, there is the following per unit loading:")
loading = network.lines_t.p0.loc[now] / network.lines.s_nom
loading.describe()

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})
fig.set_size_inches(15, 17)

network.plot(ax=ax, line_colors=abs(loading), line_cmap=plt.cm.jet, title="Line loading")

## Plotting locational marginal prices

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()})
fig.set_size_inches(20, 10)

network.plot(ax=ax, line_widths=pd.Series(0.5, network.lines.index))
plt.hexbin(network.buses.x, network.buses.y,
           gridsize=20,
           C=network.buses_t.marginal_price.loc[now],
           cmap=plt.cm.jet)

# for some reason the colorbar only works with graphs plt.plot
# and must be attached plt.colorbar

cb = plt.colorbar()
cb.set_label('Locational Marginal Price (£/MWh)')

In [ ]:
network.buses_t.marginal_price

## Plotting curtailment

In [ ]:
carrier = "Wind Onshore"

capacity = network.generators.groupby("carrier").sum().at[carrier, "p_nom"]
p_available = network.generators_t.p_max_pu.multiply(network.generators["p_nom"])
p_available_by_carrier = p_available.groupby(network.generators.carrier, axis=1).sum()
p_curtailed_by_carrier = p_available_by_carrier - p_by_carrier
p_df = pd.DataFrame({carrier + " available": p_available_by_carrier[carrier],
                     carrier + " dispatched": p_by_carrier[carrier],
                     carrier + " curtailed": p_curtailed_by_carrier[carrier]})

p_df[carrier + " capacity"] = capacity
p_df["Wind Onshore curtailed"][p_df["Wind Onshore curtailed"] < 0.] = 0.
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)
p_df[[carrier + " dispatched", carrier + " curtailed"]].plot(kind="area", ax=ax, linewidth=0)
# p_df[[carrier + " available", carrier + " capacity"]].plot(ax=ax, linewidth=0)

ax.set_xlabel("")
ax.set_ylabel("Power [MW]")
ax.legend()